<a href="https://colab.research.google.com/github/Clayton-J-Roy/clayton-roy-personal-repository/blob/Book-Suggestion-Model/vector_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
pip install langchain-community langchain-text-splitters langchain-openai langchain-chroma

In [51]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [52]:
from dotenv import load_dotenv

load_dotenv('/content/env')

True

In [53]:
import pandas as pd

books = pd.read_csv("/content/sample_data/books_cleaned.csv")

In [54]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [55]:
books["tagged_description"]

,tagged_description
0,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982 A new 'Christie for Christmas' -...
2,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897 Lewis' work on the nature of lov...
4,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...
5192,9788172235222 On A Train Journey Home To North...
5193,9788173031014 This book tells the tale of a ma...
5194,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535 This collection of the timeless ...


In [56]:
books["tagged_description"].to_csv("tagged_description.txt", sep="\n", index=False, header=False)

In [57]:
raw_documents = TextLoader("tagged_description.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Streaming output truncated to the last 5000 lines.


In [58]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gi

In [59]:
db_books = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings())

In [60]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k=10)
docs

[Document(id='6d03fd1d-8089-4b43-870a-a6cd3cac6d2c', metadata={'source': 'tagged_description.txt'}, page_content='9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'),
 Document(id='c6d28daf-46e0-4cfd-9e51-d4ce01df5bbf', metadata={'source': 'tagged_description.txt'}, page_content='9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'),
 Document(id='5e018472-4bd4-4153-8b09-2ee04fe71763', metadata={'source': 'tagged_description.txt'}, page_content='9780786808069 Children will discover the exciting world of their own backyard in this introducti

In [61]:
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...


In [62]:
def semantic_book_search(
    query: str,
    top_k: int = 10,
)-> pd.DataFrame:

  recs = db_books.similarity_search(query, k = 50)

  books_list = []

  for i in range(0, len(recs)):
    books_list += [int(recs[i].page_content.strip('"').split()[0])]

  return books[books["isbn13"].isin(books_list)]



In [63]:
semantic_book_search("a book about animals")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
85,9780030547744,0030547741,Where the Red Fern Grows,Wilson Rawls,Juvenile Fiction,http://books.google.com/books/content?id=IHpRw...,A young boy living in the Ozarks achieves his ...,2000.0,4.37,288.0,95.0,Where the Red Fern Grows: The Story of Two Dog...,9780030547744 A young boy living in the Ozarks...
707,9780140568196,0140568190,The Giraffe and the Pelly and Me,Roald Dahl;Quentin Blake,Candy,http://books.google.com/books/content?id=J7FdI...,"A Dahl story in which the giraffe, the pelican...",2001.0,3.81,32.0,16265.0,The Giraffe and the Pelly and Me,9780140568196 A Dahl story in which the giraff...
812,9780142302279,0142302279,Dirty Beasts,Roald Dahl,Juvenile Nonfiction,NaN,Poems tell the stories of a smart pig who outw...,2002.0,4.02,32.0,3953.0,Dirty Beasts,9780142302279 Poems tell the stories of a smar...
815,9780142402498,0142402494,Pippi Longstocking,Astrid Lindgren,Juvenile Fiction,http://books.google.com/books/content?id=STE5P...,Relates the antics of a rambunctious girl who ...,2005.0,4.12,160.0,140107.0,Pippi Longstocking,9780142402498 Relates the antics of a rambunct...
874,9780143039938,0143039938,The Book of Imaginary Beings,Jorge Luis Borges;Margarita Guerrero;Andrew Hu...,Fiction,http://books.google.com/books/content?id=FuNQP...,A whimsical compendium of mythological creatur...,2006.0,4.09,236.0,4809.0,The Book of Imaginary Beings,9780143039938 A whimsical compendium of mythol...
1222,9780312319731,0312319738,Dominion,Matthew Scully,Nature,http://books.google.com/books/content?id=_htG-...,Argues for responsible action in the treatment...,2003.0,4.16,448.0,1131.0,"Dominion: The Power of Man, the Suffering of A...",9780312319731 Argues for responsible action in...
1228,9780312330866,0312330863,All Things Bright and Beautiful,James Herriot,Biography & Autobiography,http://books.google.com/books/content?id=skqN2...,A Yorkshire veterinarian describes the adventu...,2004.0,4.31,378.0,60280.0,All Things Bright and Beautiful,9780312330866 A Yorkshire veterinarian describ...
1479,9780345434906,0345434900,The Real James Herriot,James Wight;Jim Wight,Biography & Autobiography,http://books.google.com/books/content?id=TrOoi...,A portrait of the best-selling author of All C...,2001.0,4.20,371.0,4178.0,The Real James Herriot: A Memoir of My Father,9780345434906 A portrait of the best-selling a...
2410,9780451457592,0451457595,The River's Gift,Mercedes Lackey,Fiction,http://books.google.com/books/content?id=qLonv...,Fifteen-year-old Ariella spends her days in th...,1999.0,3.88,128.0,1029.0,The River's Gift,9780451457592 Fifteen-year-old Ariella spends ...
3214,9780689861130,0689861133,"Moo, Baa, la la La!",Sandra Boynton,Animal sounds,http://books.google.com/books/content?id=Gz40A...,Children will love joining in and imitating th...,2004.0,4.20,14.0,28261.0,"Moo, Baa, la la La!",9780689861130 Children will love joining in an...


In [64]:
semantic_book_search("find me a book about war")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
524,9780099483472,0099483475,All Quiet on the Western Front,Erich Maria Remarque,"World War, 1914-1918",NaN,All Quiet on the Western Front is probably the...,2005.0,3.95,216.0,1018.0,All Quiet on the Western Front,9780099483472 All Quiet on the Western Front i...
572,9780140149241,0140149244,We Were the Rats,Lawson Glassop,"Tobruk, Battles of, 1941-1942",NaN,Reissue of the famous novel based on the autho...,1991.0,3.23,275.0,13.0,We Were the Rats,9780140149241 Reissue of the famous novel base...
580,9780140158762,0140158766,Chickenhawk - Back in the World,Robert Mason,Biography & Autobiography,http://books.google.com/books/content?id=R7cGA...,From the author of the bestselling autobiograp...,1994.0,4.05,400.0,166.0,Chickenhawk - Back in the World: Life After Vi...,9780140158762 From the author of the bestselli...
1199,9780312265052,0312265050,The Naked and the Dead,Norman Mailer,Fiction,http://books.google.com/books/content?id=c66GL...,Portrays the contrasting personalities and nos...,2000.0,3.94,721.0,20541.0,The Naked and the Dead: 50th Anniversary Editi...,9780312265052 Portrays the contrasting persona...
1537,9780345476098,0345476093,The Guns of August,Barbara Wertheim Tuchman,History,http://books.google.com/books/content?id=ELck9...,A definitive Pulitzer Prize-winning recreation...,2004.0,4.18,606.0,40624.0,The Guns of August: Kits for Teens,9780345476098 A definitive Pulitzer Prize-winn...
1840,9780385423755,0385423756,On the Origins of War and the Preservation of ...,Donald Kagan,History,http://books.google.com/books/content?id=dVMsA...,Analyzes history to determine why there are wa...,1996.0,4.01,573.0,494.0,On the Origins of War and the Preservation of ...,9780385423755 Analyzes history to determine wh...
2828,9780571207992,0571207995,The Wars,Timothy Findley,Fiction,http://books.google.com/books/content?id=AqnDQ...,"Robert Ross, a sensitive nineteen-year-old Can...",2001.0,3.87,218.0,6229.0,The Wars,"9780571207992 Robert Ross, a sensitive ninetee..."
2915,9780618391004,0618391002,The Lord of the Rings,Chris Smith,Literary Criticism,http://books.google.com/books/content?id=Wu8x5...,"Describes in detail, with over one thousand ph...",2003.0,4.53,218.0,18934.0,The Lord of the Rings: Weapons and Warfare,"9780618391004 Describes in detail, with over o..."
2930,9780618574810,0618574816,Tolkien And The Great War,John Garth,Biography & Autobiography,http://books.google.com/books/content?id=OODLV...,The author focuses on a little-discussed aspec...,2005.0,3.70,398.0,6167.0,Tolkien And The Great War: The Threshold Of Mi...,9780618574810 The author focuses on a little-d...
3139,9780684813219,0684813211,Achilles in Vietnam,Jonathan Shay,Psychology,http://books.google.com/books/content?id=6EEnD...,An original and groundbreaking book that exami...,1994.0,4.24,272.0,1057.0,Achilles in Vietnam: Combat Trauma and the Und...,9780684813219 An original and groundbreaking b...
